# See a Summary of All Activities

In [1]:
from IPython.display import display, Markdown
import snakemd

import fitfile
from garmindb import GarminConnectConfigManager
from garmindb.garmindb import GarminDb, Attributes, ActivitiesDb, Activities, StepsActivities, ActivityLaps, ActivityRecords
from idbutils.list_and_dict import list_not_none

from jupyter_funcs import format_number

gc_config = GarminConnectConfigManager()
db_params_dict = gc_config.get_db_params()


garmin_db = GarminDb(db_params_dict)
garmin_act_db = ActivitiesDb(db_params_dict)
measurement_system = Attributes.measurements_type(garmin_db)
unit_strings = fitfile.units.unit_strings[measurement_system]
distance_units = unit_strings[fitfile.units.UnitTypes.distance_long]

def __report_sport(sport_col, sport):
    records = Activities.row_count(garmin_act_db, sport_col, sport)
    if records > 0:
        sport_title = sport.title().replace('_', ' ')
        total_distance = Activities.get_col_sum_for_value(garmin_act_db, Activities.distance, sport_col, sport)
        if total_distance is None:
            total_distance = 0
            average_distance = 0
        else:
            average_distance = total_distance / records
        return [sport_title, records, format_number(total_distance, 1), format_number(average_distance, 1)]

doc = snakemd.new_doc()

doc.add_heading("Activities Report")
doc.add_paragraph("Analysis of all activities in the database.")

doc.add_table(
    ['Type', 'Count'],
    [
        ["Total activities", Activities.row_count(garmin_act_db)],
        ["Total Lap records", ActivityLaps.row_count(garmin_act_db)],
        ["Activity records", ActivityRecords.row_count(garmin_act_db)],
        ["Fitness activities", Activities.row_count(garmin_act_db, Activities.type, 'fitness')],
        ["Recreation activities", Activities.row_count(garmin_act_db, Activities.type, 'recreation')]
    ])

years = Activities.get_years(garmin_act_db)
years.sort()
doc.add_paragraph(f"Years with activities: {len(years)}: {years}")
sports = list_not_none(Activities.get_col_distinct(garmin_act_db, Activities.sport))
doc.add_paragraph(f"Sports: {', '.join(sports)}")
sub_sports = list_not_none(Activities.get_col_distinct(garmin_act_db, Activities.sub_sport))
doc.add_paragraph(f"SubSports: {', '.join(sub_sports)}")

sports_stats = []
for sport_name in [sport.name for sport in fitfile.Sport]:
    sport_stat = __report_sport(Activities.sport, sport_name)
    if sport_stat:
        sports_stats.append(sport_stat)
doc.add_heading("Sport Type Statistics", 3)
doc.add_table(['Sport', 'Total Activities', f'Total Distance ({distance_units})', f"Average Distance ({distance_units})"], sports_stats)

def __format_activity(activity):
    if activity:
        if activity.is_steps_activity():
            steps_activity = StepsActivities.get(garmin_act_db, activity.activity_id)
            return [activity.activity_id, activity.name, activity.type, activity.sport, format_number(activity.distance, 1), activity.elapsed_time, format_number(activity.avg_speed, 1),
                    steps_activity.avg_pace, format_number(activity.calories), format_number(activity.training_load, 1), activity.self_eval_feel, activity.self_eval_effort]
        return [activity.activity_id, activity.name, activity.type, activity.sport, format_number(activity.distance, 1), activity.elapsed_time, format_number(activity.avg_speed, 1), '',
                format_number(activity.calories), format_number(activity.training_load, 1), activity.self_eval_feel, activity.self_eval_effort]
    return ['', '', '', '', '', '', '', '', '']

activities = Activities.get_latest(garmin_act_db, 15)
rows = [__format_activity(activity) for activity in activities]
doc.add_heading("Last 15 Activities", 3)
doc.add_table(['Id', 'Name', 'Type', 'Sport', f'Distance ({distance_units})', 'Elapsed Time', f'Speed ({unit_strings[fitfile.units.UnitTypes.speed]})',
               f'Pace ({unit_strings[fitfile.units.UnitTypes.pace]})', 'Calories', 'Training Load', 'Feel', 'Effort'], rows)

rows = []
for display_activity in gc_config.display_activities():
    name = display_activity.activity_name().capitalize()
    rows.append([f'Latest {name}'] + __format_activity(Activities.get_latest_by_sport(garmin_act_db, display_activity)))
    rows.append([f'Fastest {name}'] + __format_activity(Activities.get_fastest_by_sport(garmin_act_db, display_activity)))
    rows.append([f'Slowest {name}'] + __format_activity(Activities.get_slowest_by_sport(garmin_act_db, display_activity)))
    rows.append([f'Longest {name}'] + __format_activity(Activities.get_longest_by_sport(garmin_act_db, display_activity)))

doc.add_heading("Interesting Activities", 3)
doc.add_table(['What', 'Id', 'Name', 'Type', 'Sport', f'Distance ({distance_units})', 'Elapsed Time', f'Speed ({unit_strings[fitfile.units.UnitTypes.speed]})',
               f'Pace ({unit_strings[fitfile.units.UnitTypes.pace]})', 'Calories', 'Training Load', 'Feel', 'Effort'], rows)

# smacrico added code

rows = []
for display_activity in gc_config.display_activities():
    name = display_activity.activity_name().capitalize()
    rows.append([f'Latest {name}'] + __format_activity(Activities.get_latest_by_sport(garmin_act_db, display_activity)))
    rows.append([f'Fastest {name}'] + __format_activity(Activities.get_fastest_by_sport(garmin_act_db, display_activity)))
    rows.append([f'Slowest {name}'] + __format_activity(Activities.get_slowest_by_sport(garmin_act_db, display_activity)))
    rows.append([f'Longest {name}'] + __format_activity(Activities.get_longest_by_sport(garmin_act_db, display_activity)))


doc.add_heading("Courses", 3)
courses = Activities.get_col_distinct(garmin_act_db, Activities.course_id)
doc.add_paragraph(str(courses))

display(Markdown(str(doc)))

# Activities Report

Analysis of all activities in the database.

| Type                  | Count   |
| --------------------- | ------- |
| Total activities      | 3056    |
| Total Lap records     | 5157    |
| Activity records      | 1718828 |
| Fitness activities    | 31      |
| Recreation activities | 4       |

Years with activities: 3: [2022, 2023, 2024]

Sports: other, running, cycling, walking, fitness_equipment, Other, generic, swimming, training, breathwork, transition

SubSports: generic, treadmill, casual_walking, strength_training, yoga_gym, cardio_training, flexibility_training, indoor_cycling, yoga, track, street, lap_swimming, breathing, exercise, pilates, open_water

### Sport Type Statistics

| Sport             | Total Activities | Total Distance (kilometers) | Average Distance (kilometers) |
| ----------------- | ---------------- | --------------------------- | ----------------------------- |
| Generic           | 1291             | 1.3                         | 0.0                           |
| Running           | 299              | 622.4                       | 2.1                           |
| Cycling           | 7                | 3.2                         | 0.5                           |
| Transition        | 3                | 0.0                         | 0.0                           |
| Fitness Equipment | 436              | 1.3                         | 0.0                           |
| Swimming          | 30               | 4.9                         | 0.2                           |
| Training          | 408              | 0.0                         | 0.0                           |
| Walking           | 512              | 483.1                       | 0.9                           |

### Last 15 Activities

| Id          | Name               | Type          | Sport    | Distance (kilometers) | Elapsed Time    | Speed (kph) | Pace (per kilometers) | Calories | Training Load | Feel | Effort |
| ----------- | ------------------ | ------------- | -------- | --------------------- | --------------- | ----------- | --------------------- | -------- | ------------- | ---- | ------ |
| 17412421043 | HRV                | uncategorized | generic  | 0.0                   | 00:03:10.946000 | 0.0         |                       | 28       | -             | None | None   |
| 17412364257 | Glyfada Walking    | uncategorized | walking  | 0.9                   | 00:14:23.585000 | 4.0         | 00:24:17.739357       | 101      | -             | None | None   |
| 17412064476 | Drapetsona Walking | uncategorized | walking  | 1.7                   | 00:34:36.183000 | 4.9         | 00:19:50.343352       | 125      | -             | None | None   |
| 17411919431 | F3b Monitor+HRV    | uncategorized | generic  | 0.0                   | 00:05:01.008000 | 0.0         |                       | 13       | -             | None | None   |
| 17411911225 | HRV                | uncategorized | generic  | 0.0                   | 00:07:00.245000 | 0.0         |                       | 11       | -             | None | None   |
| 17407782916 | Piraeus Walking    | uncategorized | walking  | 0.8                   | 00:22:19.828000 | 3.7         | 00:26:07.034360       | 84       | -             | None | None   |
| 17404033189 | Glyfada Walking    | uncategorized | walking  | 0.5                   | 00:21:11.484000 | 3.9         | 00:24:43.266501       | 52       | -             | None | None   |
| 17404032100 | None               | None          | training | -                     | 00:15:00.098000 | -           |                       | 44       | -             | None | None   |
| 17403551882 | Piraeus Walking    | uncategorized | walking  | 0.8                   | 00:14:29.057000 | 5.0         | 00:19:09.531598       | 62       | -             | None | None   |
| 17403340361 | F3b Monitor+HRV    | uncategorized | generic  | 0.0                   | 00:05:01.008000 | 0.0         |                       | 9        | -             | None | None   |
| 17403340260 | HRV                | uncategorized | generic  | 0.0                   | 00:03:10.985000 | 0.0         |                       | 6        | -             | None | None   |
| 17403340206 | HRV                | uncategorized | generic  | 0.0                   | 00:07:00.462000 | 0.0         |                       | 14       | -             | None | None   |
| 17399712059 | HRV                | uncategorized | generic  | 0.0                   | 00:03:10.930000 | 0.0         |                       | 16       | -             | None | None   |
| 17395499589 | HRV                | uncategorized | generic  | 0.0                   | 00:07:00.263000 | 0.0         |                       | 18       | -             | None | None   |
| 17395496881 | HRV                | uncategorized | generic  | 0.0                   | 00:03:10.972000 | 0.0         |                       | 7        | -             | None | None   |

### Interesting Activities

| What | Id | Name | Type | Sport | Distance (kilometers) | Elapsed Time | Speed (kph) | Pace (per kilometers) | Calories | Training Load | Feel | Effort |
| ---- | -- | ---- | ---- | ----- | --------------------- | ------------ | ----------- | --------------------- | -------- | ------------- | ---- | ------ |

### Courses

[None, 179203070, 280598925]